In [1]:
import numpy as np
import matplotlib.pyplot as pt
import seaborn as sns
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_style("whitegrid")


### constants 
ROW = 5*10**3 #Density kg / cubic metre
MOMINTERTIA = 1
E = 10**7 # Young's modulus
G = 2*E/3


LENGTH = 1
RADIUS = .025
AREA = np.pi * RADIUS**2
MASS = AREA * LENGTH * ROW
### Stiffness Matricies, Constant
J = np.diag(MASS*LENGTH**2/12,MASS*LENGTH**2/12,MASS*RADIUS**2/2)# M l^2 / 12 , M l^2 / 12,  M*r^2 / 2,
S = np.diag(4*G*AREA/3, 4*G*AREA/3, 2*G*AREA/3)
B = np.diag(EI1, EI2, GI3)
###
D = LENGTH # (L_i + L_i+1)/2 == L

NumberSegments = 50
DT = 2.5 * 10**(-5)

def deltaH(lists):
    temp = np.pad(lists,(1,0), 'constant', constant_values = (0,0))

    temp2 = np.pad(lists,(0,1), 'constant', constant_values = (0,0))

    return temp2 - temp
def avgAH(lists):
    temp = np.roll(x,1)
    temp = arr[1:]
    temp[0] = 0
    temp2 = lists[:-1]
    temp2[-1] = 0
    return (temp2 + temp)/2

def logMatrix(matrix):
    # log(R) = R - R.T  Skew symm matrix, must convert back to vector
    temp = matrix - matrix.T
    output = np.zeros(3)
    output[0] = temp[2][1]
    output[1] = temp[0][2]
    output[2] = temp[1][0]
    return output
def expVector(theta, vector):
    u = np.zeros((3,3))
    u[0,1] = -vector[2]
    u[0,2] = vector[1]
    u[1,2] = -vector[0]

    u[1,0] = vector[2]
    u[2,0] = -vector[1]
    u[2,1] = vector[0]

    return (np.identity(3) + np.sin(theta)*u + (1-np.cos(theta))*(u@u))


class Vertex:
    def __init__(s,i, position):
        s.I = i
        s.R = position
        s.vel = np.zeros(3)
        s.forces = np.zeros(3)

        ### Interior Vertex Value ONLY
        s.K = None
        
        s.nextVertex = None
        s.framei = None
        s.framei2 = None
        

    def setK(s):
        s.K = logMatrix(s.nextVertex.Q @ s.Q.T) / D
        
    def stepR(s):
        s.R += DT/2 * s.vel    
    def stepVel(s):
        s.vel += DT * s.forces / MASS         
    
    def stepTime(s):
        s.stepR()
        s.framei.stepQ()
        s.stepVel()
        s.framei.stepAngv()
        s.stepPos()
        s.stepQ()
        
    def AHmodified_trapz(s, lists):
        temp = np.pad(lists,(1,0), 'constant', constant_values = (0,0))
        return .5*temp - .5*roll(lists)
    
class MaterialFrame:
    def __init__(s,i):
        s.I = i
        s.Q = np.identity(3)
        s.edge = np.zeros(3)
        #s.shear = np.zeros(3) shear vecotr is always zero. tangent == d3
        s.angv = np.zeros(3)
        s.couple = np.zeros(3)
        s.dOrient = np.zeros((3,3)) # 
        ## Information about next frame and vertex
        s.nextFrame= None
        s.vertexi = None
        s.vertexi1 = None
        
    def setEdges(s):
        s.edge = s.vertexi1.R - s.vertexi.R
    def changeInOrientation(s):
        for i in enumerate(s.dOrient):
            s.dOrient[i] = np.cross(s.Q.T @ s.angv,s.Q[i])
            
    def stepQ(s):
        s.Q *= expVector(DT/2,s.angv)
        
    def stepAngv(s):
        s.angv += DT * s.couple / s.MOINERTIA
        
#Time = 1000;
#positions = np.zeros((3,Time))


class Snake:
    def __init__(s):

        s.Vertices = [None]*(NumberSegments+1)
        s.MatFrames = [None] * (NumberSegments)
        for i in range(s.NumberSegments+1):
            s.Vertices[i] = Vertex(i,np.array((i,0,0)))
            if(i <= s.NumberSegments):
                s.MatFrames[i] = MaterialFrame(i)
        for i in enumerate(s.MatFrames):
            s.MatFrames[i].vertexi = s.Vertices[i]
            s.MatFrames[i].vertexi1 = s.Vertices[i+1]
            if(i!= len(s.MatFrames)):
                s.MatFrames[i].nextFrame = s.MatFrames[i+1]
        for i in enumerate(s.Vertices):
            if(i< len(s.Vertices)):
                s.Vertices[i].nextVertex = s.Vertices[i+1]
                s.Vertices[i].framei = s.MatFrames[i]
                if(i< len(s.Vertices)-1):
                    s.Vertices[i].framei1 = s.MatFrames[i+1]
           
    

def main():
    x = np.array((1,1,3,5,6,7))
    print(x)
    b = np.roll(x,-1)
    temp = np.pad(x,(1,0), 'constant', constant_values = (0,0))
    print(temp)

    
main()

SyntaxError: invalid syntax (<ipython-input-1-9322f9ea2f06>, line 21)